In [41]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn,rnn_cell
from tensorflow.contrib import rnn

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True) #one hot is useful for multiclass classification

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [28]:
hm_epochs = 3
n_classes = 10 #no of classes [0-9] 
batch_size = 128

chunk_size = 28
n_chunks = 28
rnn_size = 128

In [29]:
#height x width
x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

In [48]:
##################### DEFINING THE COMPUTATIONAL GRAPH ##############################

#(input *weights)+biases
def rnn_model(x):
    
    layer = {'Weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x,[1,0,2])
    x = tf.reshape(x,[-1,chunk_size])
    x = tf.split(x,n_chunks,0)
    
    lstm = rnn.BasicLSTMCell(rnn_size, state_is_tuple=True)
    outputs, states = rnn.static_rnn(lstm, x, dtype=tf.float32)
    
    output = tf.add(tf.matmul(outputs[-1], layer['Weights']), layer['biases'])
 
    return output

In [49]:

################ TRAINING THE NEURAL NETWORK ########################
def train_network(x): 
    
    prediction = rnn_model(x)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = prediction, labels = y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #feed_forwards + backprops
    hm_epochs = 10
    
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss= 0 
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))
                
                _ , c= sess.run([optimizer,cost], feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss += c
            print("Epoch",epoch, 'completed out of',hm_epochs,'loss: ', epoch_loss)
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ', accuracy.eval({x:mnist.test.images.reshape((-1,n_chunks,chunk_size)), y: mnist.test.labels}))
            

In [50]:
train_network(x)

Epoch 0 completed out of 10 loss:  203.03490328
Epoch 1 completed out of 10 loss:  58.6574071348
Epoch 2 completed out of 10 loss:  39.68665832
Epoch 3 completed out of 10 loss:  31.0281607411
Epoch 4 completed out of 10 loss:  25.16168012
Epoch 5 completed out of 10 loss:  22.1253910875
Epoch 6 completed out of 10 loss:  18.6101591513
Epoch 7 completed out of 10 loss:  16.4461815127
Epoch 8 completed out of 10 loss:  14.1391159787
Epoch 9 completed out of 10 loss:  13.4308426724
Accuracy:  0.9858
